# Notebook para carregar a base CNPJ da Receita Federal e realizar Buscas de CNAE

In [1]:
# Importar Bibliotecas
import polars as pl
import time
import os
import gc

# Marca o tempo de início
start_time = time.time()

# Defina o caminho da pasta que contém os arquivos CSV
directory_path = "Base/"

# Defina os nomes das colunas
column_names = ["column01", "column02", "column03", "column04", "column05", "column06", "column07", "column08", "column09", 
                "column10", "column11", "column12", "column13", "column14", "column15", "column16", "column17", "column18", 
                "column19", "column20", "column21", "column22", "column23", "column24", "column25", "column26", "column27", 
                "column28", "column29", "column30"]

# Inicialize uma lista para armazenar os DataFrames
dfs = []

# Função para criar um esquema de colunas onde todas são do tipo string
def create_schema(column_names):
    return {col: pl.Utf8 for col in column_names}

# Crie o esquema
schema = create_schema(column_names)

# Liste todos os arquivos CSV no diretório
for file_name in os.listdir(directory_path):
    if file_name.endswith(".csv"):
        file_path = os.path.join(directory_path, file_name)
        # Carregue o CSV
        df = pl.read_csv(file_path, separator=';', encoding='latin1', ignore_errors=True, schema=schema)
        # Mudar nome das colunas
        df.columns = column_names
        # Adicione o DataFrame à lista
        dfs.append(df)

# Concatene todos os DataFrames em um único DataFrame
final_df = pl.concat(dfs)

# Detela o df
del df

# Forçar a coleta de lixo para liberar a memória
gc.collect()

# Marca o tempo de término
end_time = time.time()

# Calcula o tempo de execução
execution_time = end_time - start_time

# Para verificar a memória utilizada
print(final_df.shape)
print(f"Tempo de execução: {execution_time} segundos")

(60944816, 30)
Tempo de execução: 75.10680222511292 segundos


In [3]:
############# TRATAMENTO DE DADOS #############

# Lista de colunas a serem removidas    
colunas_para_remover = [
        'column06', 'column08', 'column09', 'column10', 'column14', 'column15', 'column16', 'column17', 
        'column18', 'column19',  'column20', 'column21', 'column22', 'column23', 'column24', 'column25', 
        'column26', 'column27', 'column28', 'column29', 'column30']

# Deletando Colunas
final_df = final_df.drop(colunas_para_remover)

# Concatenar duas colunas (por exemplo, "column03" e "column04")
final_df = final_df.with_columns((final_df["column01"]  + final_df["column02"] + final_df["column03"]).alias("CNPJ"))

# Lista de colunas a serem removidas    
colunas_para_remover = [
        'column01', 'column02', 'column03','column04']

# Deletando Colunas
final_df = final_df.drop(colunas_para_remover)

# Renomeando colunas
final_df = final_df.rename({"column05": "Razao_Social"})
final_df = final_df.rename({"column07": "Data_Situacao"})
final_df = final_df.rename({"column11": "Data_Abertura"})
final_df = final_df.rename({"column12": "CNAE_Principal"})
final_df = final_df.rename({"column13": "CNAE_Secundario"})

In [5]:
############# REALIZAR PESQUISA POR CNPJ #############

# Filtra linhas onde o valor da coluna "CNPJ" é igual a "83770461000170"
resultados = final_df.filter(pl.col("CNPJ") == "16564523000109")
resultados

Razao_Social,Data_Situacao,Data_Abertura,CNAE_Principal,CNAE_Secundario,CNPJ
str,str,str,str,str,str
"""TRENTO""","""20140520""","""20120627""","""6462000""","""""","""16564523000109"""
